# User Interface 

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly_usmap import usmap

## 1. Set up options for Dropdown 

In [ ]:
rank = pd.read_csv('rank_file.csv')

# Create a list of indicators 
available = list(rank.columns.values)
for i in ['Unnamed: 0','City','State','Population','Natural_total_rank', 'Human_related_rank', 
          'Economy_rank', 'Tertiary_Rank', 'Latitude', 'Longitude']:
    available.remove(i)
    
# Create a list of labels for dropdown
labels = ['Fewer Crimes','More Hospitals','Early Education Options',
          'University Options','Air Quality','Water Quality',
           'Fewer Toxics','Fewer Hazardous Particles','Green Coverage',
           'Employment Rate','Sales Revenue','Income','Tuition Affordability',
          'Bars','Restaurants','Museums','Libraries','Parks','Top Restaurants']

# Put available and labels in a two-dimensional list
pairs = [available,labels]

## 2. Set up layout for User Interface

In [ ]:
app = dash.Dash()
# Get DataFrame value
import UI_setup
app.layout = UI_setup.layout_setup(pairs)



## 3.Set up for default graph

In [ ]:
@app.callback(
    dash.dependencies.Output('Total-graphic', 'figure'),
    [dash.dependencies.Input('Total', 'n_clicks')]
)
def total_graph(Total):
    if(Total is None):
        return
    #Total general rank
    return {
           usmap(rank)    
        }

In [ ]:
@app.callback(
    dash.dependencies.Output('Human-graphic', 'figure'),
    [dash.dependencies.Input('Hrank', 'n_clicks')]
)
def total_graph(Hrank):
    if(Hrank is None):
        return
    #Total general rank
    return {
           usmap(rank, 'human')    
        }

In [ ]:
@app.callback(
    dash.dependencies.Output('Natural-graphic', 'figure'),
    [dash.dependencies.Input('Nrank', 'n_clicks')]
)
def total_graph(Nrank):
    if(Nrank is None):
        return
    #Total general rank
    return {
           usmap(rank, 'natural')    
        }

In [ ]:
@app.callback(
    dash.dependencies.Output('Economic-graphic', 'figure'),
    [dash.dependencies.Input('Erank', 'n_clicks')]
)
def total_graph(Erank):
    if(Erank is None):
        return
    #Total general rank
    return {
           usmap(rank, 'economy')    
        }

In [ ]:
@app.callback(
    dash.dependencies.Output('Tertiary-graphic', 'figure'),
    [dash.dependencies.Input('Trank', 'n_clicks')]
)
def total_graph(Trank):
    if(Trank is None):
        return
    #Total general rank
    return {
           usmap(rank, 'tertiary')    
        }

## 4.Set up for user call

In [ ]:
@app.callback(
    dash.dependencies.Output('User-graphic', 'figure'),
    [dash.dependencies.Input('Search', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value'),
     dash.dependencies.State('Third-care', 'value'),
     dash.dependencies.State('Fourth-care', 'value'),
     dash.dependencies.State('Fifth-care', 'value'),
    ]
)
    
def update_graph(Search, First_care, Second_care, Third_care, Fourth_care, Fifth_care):
    df = UI_setup.newdf(rank, First_care, Second_care, Third_care, Fourth_care, Fifth_care)
    
    limits = [(0,10),(10,20),(20,30),(30,40),(40,50)]
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
    cities = []


    for i in range(len(limits)):
        lim = limits[i]
        df_sub = df[lim[0]:lim[1]]
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['reverse_rank']*15,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0],lim[1]) )
        cities.append(city)

        layout = dict(
            title = 'Your Dream City Results',
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )
        fig = dict( data=cities, layout=layout )
        
    return {   
        plotly.offline.plot( fig, validate=False )     
    }

## 5. Running

In [ ]:
app.run_server()